<img src="http://www.ubu.es/sites/default/files/portal_page/images/logo_color_2l_dcha.jpg" height="150" width="150" align="right"/>
# Filtro Colaborativo basado en usuarios
[Nacho Santos](www.nacho.santos.name)

In [1]:
import numpy as np

## Collaborative Filtering (CF)
El filtrado colaborativo (**Collaborative Filtering**) es una técnica ampliamente utilizada como sistema de recomendación. Un CF utiliza una matriz de utilidad (e.g. ratings) de por un conjunto de $m$ usuarios $U=\{u_1,u_2,...,u_m\}$ sobre un conjunto de $n$ productos $I=\{i_1,i_2,...,i_n\}$ para generar predicciones (recomendaciones).<br>
En este notebook se explica la aproximación **User-based CF**: la base de la recomendación son los usuarios más próximos a quien se quiere recomendar.

### Datos de ejemplo
Ejemplo de críticas de usuarios a películas

In [2]:
# Diccionario de usuarios y valoraciones
critics={'UserA':{'Star Wars VII':4,'Ocho apellidos vascos':3,'Marte':5},
         'UserB':{'Batman v Superman':5,'Ocho apellidos vascos':4},
         'UserC':{'Star Wars VII':5,'Batman v Superman':4,'Ocho apellidos vascos':2},
         'UserD':{'Star Wars VII':2,'Batman v Superman':4,'Marte':3},
         'UserE':{'Star Wars VII':3,'Batman v Superman':4,'Ocho apellidos vascos':5}}

In [3]:
# Función para pasar del diccionario a las matrices Y (ratings), R(películas vistas)
def dic_ratings2arrays(dic):
    usuarios = len(dic)
    peliculas = 0
    
    movies = []
    users = []
    
    for usuario in dic:
        users.append(usuario)
        for pelicula in dic[usuario].keys():
            if not pelicula in movies:
                movies.append(pelicula)
    #movies = np.unique(movies)
    peliculas = len(movies)
    
    Y = np.zeros((usuarios,peliculas))
    R = np.zeros((usuarios,peliculas))
    
    fila = -1
    for usuario in dic:
        fila += 1
        val_R = []
        val_Y = []
        for pelicula in movies:
            if pelicula in dic[usuario].keys():
                val_R.append(1)
                val_Y.append(dic[usuario][pelicula])
            else:
                val_R.append(0)
                val_Y.append(0)
        R[fila] = np.array(val_R)
        Y[fila] = np.array(val_Y)
    

    return (Y,R,users,movies)

In [4]:
(Y,R,users,movies)=dic_ratings2arrays(critics)
print('Matrix of ratings Y\n',Y)
print('Matrix R\n',R)
print('Users\n',users)
print('Movies\n',movies)

Matrix of ratings Y
 [[4. 3. 5. 0.]
 [0. 4. 0. 5.]
 [5. 2. 0. 4.]
 [2. 0. 3. 4.]
 [3. 5. 0. 4.]]
Matrix R
 [[1. 1. 1. 0.]
 [0. 1. 0. 1.]
 [1. 1. 0. 1.]
 [1. 0. 1. 1.]
 [1. 1. 0. 1.]]
Users
 ['UserA', 'UserB', 'UserC', 'UserD', 'UserE']
Movies
 ['Star Wars VII', 'Ocho apellidos vascos', 'Marte', 'Batman v Superman']


### Similitud
Existen diferentes formas de medir la similitud (proximidad o distancia) entre usuarios, o entre productos. Por ejemplo, dados dos usuarios **u** y **v**, se puede utilizar los vectores de ratings de aquellos productos que han sido evaluados por ambos usuarios para calcular:
* La **distancia euclídea** entre **u** y **v** en el espacio multidimensional que forman los productos evaluados por ambos. El problema fundamental de este tipo de medidas es que depende del rango de valores utilizados en los ratings de los usuarios (que pueden variar de un problema a otro).
* La **similitud de coseno**, supone que los vectores representan a los usuarios en el espacio multidimensional de productos (evaluados por ambos) y calcula el [coseno del ángulo](https://en.wikipedia.org/wiki/Cosine_similarity) que forman los vectores. La medida varía entre (-1,1), siendo 1 cuando el ángulo es 0º (coinciden), 0 cuando el ángulo es 90º y -1 cuando el ángulo es -180º.
* La **correlación de Pearson**, los vectores de los usuario pueden interpretarse como una nube de puntos en el espacio multidimensional de ratings, y de esta forma calcularse el **coeficiente de correlación lineal** como medida de similitud. En este caso la similitud está normalizada entre (-1,1), siendo positiva cuando la correlación es positiva y negativa en caso contrario.

In [5]:
# Correlación de Pearson (sobre coincidencia en el espacio de valoraciones)
def pearson_similarity(Y,R,a,b):
    R_a = R[a,:]
    R_b = R[b,:]
    Y_a = Y[a,:]
    Y_b = Y[b,:]
    
    comunes = R_a * R_b
    posiciones_comunes = np.where(comunes>0)
    
    numerador = np.sum((Y_a[posiciones_comunes]-np.mean(Y_a[R_a>0]))*(Y_b[posiciones_comunes]-np.mean(Y_b[R_b>0])))
    denominador = np.sqrt(np.sum((Y_a[posiciones_comunes]-np.mean(Y_a[R_a>0]))**2)) * np.sqrt(np.sum((Y_b[posiciones_comunes]-np.mean(Y_b[R_b>0]))**2))

    pcorr = numerador / denominador
    
    return pcorr

### Similitud entre usuarios

In [6]:
# Similitud entre dos usuarios
for (u1,u2) in [(0,2)]:
    print('similarity between', users[u1], 'and', users[u2])
    print('pearson: %0.4f' % pearson_similarity(Y,R,u1,u2))

similarity between UserA and UserC
pearson: 0.7809


In [7]:
# Similitud entre todos los usuarios
for i in range(len(users)):
    for j in range(len(users)):
        if i != j:
            print('similarity between', users[i], 'and', users[j])
            print('pearson: %0.4f' % pearson_similarity(Y,R,i,j))

similarity between UserA and UserB
pearson: 1.0000
similarity between UserA and UserC
pearson: 0.7809
similarity between UserA and UserD
pearson: 0.0000
similarity between UserA and UserE
pearson: -0.7071
similarity between UserB and UserA
pearson: 1.0000
similarity between UserB and UserC
pearson: 0.8321
similarity between UserB and UserD
pearson: 1.0000
similarity between UserB and UserE
pearson: -0.7071
similarity between UserC and UserA
pearson: 0.7809
similarity between UserC and UserB
pearson: 0.8321
similarity between UserC and UserD
pearson: -0.5145
similarity between UserC and UserE
pearson: -0.9820
similarity between UserD and UserA
pearson: 0.0000
similarity between UserD and UserB
pearson: 1.0000
similarity between UserD and UserC
pearson: -0.5145
similarity between UserD and UserE
pearson: 0.7071
similarity between UserE and UserA
pearson: -0.7071
similarity between UserE and UserB
pearson: -0.7071
similarity between UserE and UserC
pearson: -0.9820
similarity between User

### Ejemplo: predecir el rating de 'UserC' a la película 'Marte'
1. Calcular la matriz de similitud entre usuarios (esto solo es necesario hacerlo una vez).
2. Encontrar los k usuarios más próximos a 'UserC' que evaluaron la película 'Marte' (e.g. similitud >0)
3. Calcular la predicción del rating de 'UserC' a la película 'Marte' usando la fórmula:

$$\hat{r}_{u,i}=\bar{r}_{u}+\frac{\sum_{v\in{U}}(r_{v,i}-\bar{r}_{v})sim_{u,v}}{\sum_{v\in{U}} \mid sim_{u,v}\mid}$$
<div class="alert alert-block alert-info">
La estimación del rating del usuario u a la película i es igual al valor medio de raings del usuario u más <b>la media ponderada de los ratings (normalizadas) de los usuarios próximos a u que vieron la película i</b>, donde los pesos son proporcionales a la similitud entre usuarios
</div>

**1 Calcular la matriz de similitud entre usuarios**

In [8]:
# Calcular la matriz de similitud entre usuarios
def user_similarity_matrix(Y,R):
    
    matrix = np.zeros((len(users),len(users)))
    
    for i in range(len(users)):
        for j in range(len(users)):
            matrix[i][j] = pearson_similarity(Y,R,i,j)
    return matrix

In [9]:
user_sim=user_similarity_matrix(Y,R)
print('User similarity matrix\n',user_sim)

User similarity matrix
 [[ 1.          1.          0.78086881  0.         -0.70710678]
 [ 1.          1.          0.83205029  1.         -0.70710678]
 [ 0.78086881  0.83205029  1.         -0.51449576 -0.98198051]
 [ 0.          1.         -0.51449576  1.          0.70710678]
 [-0.70710678 -0.70710678 -0.98198051  0.70710678  1.        ]]


**2 Encontrar los k usuarios más próximos a 'UserC' que evaluaron la película 'Marte'. Supondremos que proximidad es equivalente a tener una similitud estrictamente positiva**

In [10]:
sim_c=user_sim[users.index("UserC")]
usuariosSimilares = np.where(sim_c > 0)
vistoMarte = R[:,movies.index("Marte")]
vistoMarte = np.where(vistoMarte > 0)
usuariosSimilares = np.intersect1d(usuariosSimilares,vistoMarte)

**3 Calcular la predicción del rating de 'UserC' a la película 'Marte'**

In [11]:
media_ponderada = 0
numerador = 0
denominador = 0
resultado = 0
media_todos = []

for i in users:
    media_todos.append(np.mean(Y[users.index(i), R[users.index(i),:]>0]))

#Primera parte
media_u = media_todos[users.index("UserC")] 

#Segunda parte
for i in usuariosSimilares:
    rating_peli = Y[i,movies.index("Marte")]
    rating_v = media_todos[i]
    similitud = user_sim[users.index("UserC"),i]
    numerador = np.sum(rating_peli-rating_v)*similitud
    denominador = np.sum(np.abs(similitud))
    resultado += numerador/denominador

#Parte final suma todo
resultado += media_u
print(resultado)




4.666666666666666


### Referencias
* Ekstrand, M. D., Riedl, J. T., & Konstan, J. A. (2011). Collaborative filtering recommender systems. Foundations and Trends in Human-Computer Interaction, 4(2), 81-173. Section 2.1,2.2 and 2.3
